In [49]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv
import pandas as pd

In [50]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


# TABLE DEFINITION

In [16]:
#calculates the pearson correlation and cosine similarity while accounting for the corner cases
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import warnings

def find_pearson(v1,v2):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        if np.array_equal(v1,v2):
            return 1
        elif abs(sum(v1 - v2)) == v1.size:
            return -1
        else:
            #perform the pearson correlation
            corr_conversion, p_value_conversion = pearsonr(v1, v2)
            return corr_conversion

def find_cosine(v1,v2):
    if np.array_equal(v1,v2):
        return 1
    elif abs(sum(v1 - v2)) == v1.size:
        return 0
    else:
        v1 = v1.reshape(1,len(v1))
        v2 = v2.reshape(1,len(v2))
        return cosine_similarity(v1, v2)[0][0]

def jaccard_similarity(x,y):

    """ returns the jaccard similarity between two lists """

    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)
    

In [89]:
# v1 = np.array([1,1,1,1,1])
# v2 = np.array([0,0,0,0,0])
# print(find_cosine(v1,v2))
# v1 = np.array([0,0,0,0,0])
# v2 = np.array([0,0,0,0,0])
# print(find_cosine(v1,v2))
v1 = np.array([0,1,0,0,0]*100)
v2 = np.array([1,1,1,1,0]*100)
print(find_cosine(v1,v2))
print(find_pearson(v1,v2))
# v1 = v1.reshape(1,len(v1))
# v2 = v2.reshape(1,len(v2))
# print(cosine_similarity(v1, v2))
print("cohen = " + str(sklearn.metrics.cohen_kappa_score(v1,v2)))

#can be how to get the number of matches (but not tell what type)
start_time = time.time()
similar = np.dot((np.vstack((v1,v2)).T),np.array([-0.5,0.5])) == 0
print(f"total time = {time.time()-start_time}")

# start_time = time.time()
# greater = sum(v1 > v2
# less = v1 < v2
# equal = v1==v2 
# ones = v1[v1==v2] == 1
# print(f"total time = {time.time()-start_time}")
# #print(greater)


# different = similar == False
# higher np.dot((np.vstack((v1,v2)).T),np.array([-0.5,0.5])) == 0
# print(similar)
# print(different)
#represents the mismatches
#b,c = 

# print(jaccard_similarity(v1,v2)) #non-correlation based similarity measurement, alright

0.5
0.24999999999999994
cohen = 0.11764705882352944
total time = 0.0007026195526123047


In [154]:
v1 = np.array([1,0,0,0]*100)
v2 = np.array([1,1,1,0]*100)

np.vstack((v1,1-v1))
np.vstack((v2,1-v2))

start_time = time.time()
print(np.dot(np.vstack((v1,1-v1)),np.vstack((v2,1-v2)).T))
print(f"total time = {time.time()-start_time}")

start_time = time.time()
a = np.dot(v1,v2)
b = np.dot(1-v1,v2)
c = np.dot(v1,1-v2)
d = np.dot(1-v1,1-v2)
(a + d)/(a + b + c + d)
print(f"total time = {time.time()-start_time}")
print(a,b,c,d)

start_time = time.time()
sklearn.metrics.jaccard_similarity_score(v1,v2)
print(f"total time = {time.time()-start_time}")


[[100   0]
 [200 100]]
total time = 0.0013179779052734375
total time = 0.0009391307830810547
100 200 0 100
total time = 0.003767251968383789


In [132]:
def tarantula(v1,v2):
    a = np.dot(v1,v2)
    b = np.dot(1-v1,v2)
    c = np.dot(v1,1-v2)
    d = np.dot(1-v1,1-v2)
    #print(a,b,c,d)

    return (a*(c+d))/(c*(a+b))


def jaccard_bac(v1,v2):
    a = np.dot(v1,v2)
    b = np.dot(1-v1,v2)
    c = np.dot(v1,1-v2)
    d = np.dot(1-v1,1-v2)
    #print(a,b,c,d)

    return (a + d)/(a + b + c + d)

def jaccard_bac_stacked(v1,v2):
    v_total = np.dot()
    
    a = np.dot(v1,v2)
    b = np.dot(1-v1,v2)
    c = np.dot(v1,1-v2)
    d = np.dot(1-v1,1-v2)
    #print(a,b,c,d)

    return (a + d)/(a + b + c + d)

In [136]:
start_time = time.time()
v1 = np.array([0,0,0,0,1]*100)
v2 = np.array([0,0,0,0.1,0.1]*100)
#print(tarantula(v1,v2))
print("pearson = "+str(find_pearson(v1,v2)))
print("cosine = "+str(find_cosine(v1,v2)))
print("jaccard_bac = " + str(jaccard_bac(v1,v2)))
print("jaccard_similarity = " + str(jaccard_similarity(v1,v2)))
print("jaccard_similarity_score sklearn = " + str(sklearn.metrics.jaccard_similarity_score(v1,v2)))

print(f"total time = {time.time()-start_time}")

pearson = 0.6123724356957947
cosine = 0.7071067811865475
jaccard_bac = 0.8
jaccard_similarity = 0.3333333333333333


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [93]:
dir(sklearn.metrics)

['SCORERS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'accuracy_score',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'auc',
 'average_precision_score',
 'balanced_accuracy_score',
 'base',
 'brier_score_loss',
 'calinski_harabaz_score',
 'check_scoring',
 'classification',
 'classification_report',
 'cluster',
 'cohen_kappa_score',
 'completeness_score',
 'confusion_matrix',
 'consensus_score',
 'coverage_error',
 'davies_bouldin_score',
 'euclidean_distances',
 'explained_variance_score',
 'f1_score',
 'fbeta_score',
 'fowlkes_mallows_score',
 'get_scorer',
 'hamming_loss',
 'hinge_loss',
 'homogeneity_completeness_v_measure',
 'homogeneity_score',
 'jaccard_similarity_score',
 'label_ranking_average_precision_score',
 'label_ranking_loss',
 'log_loss',
 'make_scorer',
 'matthews_corrcoef',
 'mean_absolute_error',
 'mean_squared_error',
 'mean_squared_log_error',
 'median_absol

In [23]:
import math
myArray = np.array([1.2,0.2,0.6,0,2.2,0,.2,0,0.1,2.9,0.1])
print("myArray  = " + str(myArray))
myArray[myArray>1] = 1
sum(np.ceil(myArray))
# myArray[myArray>1] = 1
# print("myArray  = " + str(myArray))
# print(np.ceil(myArray))
# sum(np.ceil(myArray))

myArray  = [1.2 0.2 0.6 0.  2.2 0.  0.2 0.  0.1 2.9 0.1]


8.0

In [4]:
@schema
class ContactCorrelation(dj.Computed):
    definition="""
    -> ta3p100.Segment
    segment_b :bigint unsigned #id of the postsynaptic neuron
    ---
    n_seg_a =              :bigint unsigned #n_presyns contacting onto segment_id
    n_seg_b =              :bigint unsigned #n_presyns contacting onto segment_b
    n_seg_shared           :bigint unsigned #n_presyns contacting onto both segment_id and segment_b
    n_seg_union            :bigint unsigned #n_presyns contacting either segment_id or segment_b
    n_seg_shared_converted :bigint unsigned #n_presyns contacting onto both and converting on at least 1 postsyn
    n_seg_a_converted      :bigint unsigned #n_presyns contacting onto both and converting on postsyna a
    n_seg_a_converted_prop :float           #proportion of n_presyns contacting onto both which convert at least onto postsyna a
    n_seg_b_converted      :bigint unsigned #n_presyns contacting onto both and converting on postsyna b
    n_seg_b_converted_prop :float           #proportion of n_presyns contacting onto both which convert at least onto postsyna b
    binary_conversion_pearson=null :float   #pearson correlation for binary n_synapse/n_contact rate
    binary_conversion_cosine=null :float    #cosine similarity correlation for binary n_synapse/n_contact rate
    conversion_pearson=null :float          #Pearson correlation for n_synapse/n_contact rate
    conversion_cosine=null :float           #cosine similarity for n_synapse/n_contact rate
    density_pearson=null :float             #Pearson correlation for n_synapse/postsyn_length rate
    density_cosine=null :float              #cosine similarity for n_synapse/postsyn_length rate
    syn_volume_mean_pearson=null :float     #Pearson correlation for mean of synaptic volume
    syn_volume_mean_cosine=null :float      #cosine similarity for mean of synaptic volume
    syn_density_pearson=null :float         #Pearson correlation for n_synapses*synapse_sizes_mean/postsyn_length rate
    syn_density_cosine=null :float          #cosine similarity for n_synapses*synapse_sizes_mean/postsyn_length rate
    binary_conversion_pearson_converted=null :float   #pearson correlation for binary n_synapse/n_contact rate for axon group with at least 1 conversion
    binary_conversion_cosine_converted=null :float    #cosine similarity correlation for binary n_synapse/n_contact rate for axon group with at least 1 conversion
    conversion_pearson_converted=null :float          #Pearson correlation for n_synapse/n_contact rate for axon group with at least 1 conversion
    conversion_cosine_converted=null :float           #cosine similarity for n_synapse/n_contact rate for axon group with at least 1 conversion
    density_pearson_converted=null :float             #Pearson correlation for n_synapse/postsyn_length rate for axon group with at least 1 conversion
    density_cosine_converted=null :float              #cosine similarity for n_synapse/postsyn_length rate for axon group with at least 1 conversion
    syn_volume_mean_pearson_converted=null :float     #Pearson correlation for mean of synaptic volume for axon group with at least 1 conversion
    syn_volume_mean_cosine_converted=null :float      #cosine similarity for mean of synaptic volume for axon group with at least 1 conversion
    syn_density_pearson_converted=null :float         #Pearson correlation for n_synapses*synapse_sizes_mean/postsyn_length rate for axon group with at least 1 conversion
    syn_density_cosine_converted=null :float          #cosine similarity for n_synapses*synapse_sizes_mean/postsyn_length rate for axon group with at least 1 conversion

    """

    key_source = ta3p100.Segmentation & ta3p100.CurrentSegmentation
    
    def make(self,key):
        #Retrieves the PrePost table that will be using in an all in one insertion (MAY HAVE TO ADJUST FOR BIGGER DATA SETS IN FUTURE)
        prepost_data = ta3p100.ContactPrePost2.proj("postsyn","total_contact_conversion",
                "total_contact_density","total_synapse_sizes_mean",
                syn_density="(total_n_synapses*total_synapse_sizes_mean)/total_postsyn_length",
                presyn="segment_id").fetch()
        df = pd.DataFrame(prepost_data)

        #gets all the combinations of postsyn-postsyn without any repeats
        targets = (dj.U("postsyn") & ta3p100.Contact).proj(segment_id="postsyn") - ta3p100.SegmentExclude
        info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
        segment_pairs = info.fetch()
        
        total_correlations = []

        for postsyn1,postsyn2 in segment_pairs:

            start_time = time.time()



            #print("postsyn1 = " + str(postsyn1))
            #print("postsyn2 = " + str(postsyn2))

            #get all of the rows with postsyn 1 and 2
            df_1 = df[df["postsyn"]==postsyn1]
            df_2 = df[df["postsyn"]==postsyn2]

            #find the number of presyns for each

            #how you restrict a table by the values in the other table
            df_1_common = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])
            df_2_common = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])


            ###########------------------------------------------------------###########
            #need to get the common axons that have at least one converted contact on one of the postsyns
            """pseudocode
            #get the conversion rates for both tables
            #add them up
            #get the indices that are greater than 0
            #get the presyn ids that match those rows
            #further restrict both groups by those ids
            """

            #get both of their conversion rates
            test_1_conv = df_1_common["total_contact_conversion"].to_numpy()
            test_2_conv = df_2_common["total_contact_conversion"].to_numpy()

            test_1_presyn = df_1_common["presyn"].to_numpy()
            new_presyns = test_1_presyn[(test_1_conv + test_2_conv) > 0]

            df_1_common_converted = df_1_common[df_1_common["presyn"].isin(new_presyns)]
            df_2_common_converted = df_2_common[df_2_common["presyn"].isin(new_presyns)]
            ###########------------------------------------------------------###########

            dict_segmenation=2
            dict_segment_id=postsyn1
            dict_segment_b=postsyn2


            #finds the number of segments, shared_segments and union segments
            n_seg_a = df_1.shape[0]
            n_seg_b = df_2.shape[0]
            n_seg_shared = df_1_common.shape[0]
            n_seg_shared_converted = df_1_common_converted.shape[0]
            n_seg_union = n_seg_a + n_seg_b - n_seg_shared
            
            
            #get the number and proportion on presyns that convert onto each segment inside the converted axon group
            test_1_conv[test_1_conv>1] = 1
            n_seg_a_converted = sum(np.ceil(test_1_conv))
            test_2_conv[test_2_conv>1] = 1
            n_seg_b_converted = sum(np.ceil(test_2_conv))
            
            n_seg_a_converted_prop = n_seg_a_converted/n_seg_shared_converted
            n_seg_b_converted_prop = n_seg_b_converted/n_seg_shared_converted
     

            #initialize the dictionary that will be saved:
            corr_dict = dict(segmentation=2,segment_id=postsyn1,
                                          segment_b=postsyn2,
                                          n_seg_a=n_seg_a,
                                            n_seg_b=n_seg_b,
                                            n_seg_shared=n_seg_shared,
                                            n_seg_shared_converted=n_seg_shared_converted,
                                            n_seg_union=n_seg_union,
                                            n_seg_a_converted=n_seg_a_converted,
                                            n_seg_b_converted=n_seg_b_converted,
                                            n_seg_a_converted_prop=n_seg_a_converted_prop,
                                            n_seg_b_converted_prop=n_seg_b_converted_prop)


            #initialize the variables that need to be set in the dictionary


            #ones that are set by 1st group
            binary_conversion_pearson = np.NaN
            binary_conversion_cosine = np.NaN
            conversion_pearson = np.NaN
            conversion_cosine = np.NaN
            density_pearson = np.NaN
            density_cosine = np.NaN
            syn_volume_mean_pearson = np.NaN
            syn_volume_mean_cosine = np.NaN
            syn_density_pearson = np.NaN
            syn_density_cosine = np.NaN

            #ones that are set by 2nd group
            binary_conversion_pearson_converted = np.NaN
            binary_conversion_cosine_converted = np.NaN
            conversion_pearson_converted = np.NaN
            conversion_cosine_converted = np.NaN
            density_pearson_converted = np.NaN
            density_cosine_converted = np.NaN
            syn_volume_mean_pearson_converted = np.NaN
            syn_volume_mean_cosine_converted = np.NaN
            syn_density_pearson_converted = np.NaN
            syn_density_cosine_converted = np.NaN




            if (not df_1_common.to_numpy().any()) or (not df_2_common.to_numpy().any()):
                #total_correlations.append(corr_dict)
                pass

            else:
                df_1_common_conversion = df_1_common["total_contact_conversion"].to_numpy()
                df_2_common_conversion = df_2_common["total_contact_conversion"].to_numpy()

                df_1_common_binary_conversion = np.copy(df_1_common_conversion)
                df_2_common_binary_conversion = np.copy(df_2_common_conversion)


                df_1_common_binary_conversion[df_1_common_binary_conversion>0] = 1.0
                df_2_common_binary_conversion[df_2_common_binary_conversion>0] = 1.0

                df_1_common_density = df_1_common["total_contact_density"].to_numpy()
                df_2_common_density = df_2_common["total_contact_density"].to_numpy()

                df_1_common_synaptic_size = df_1_common["total_synapse_sizes_mean"].to_numpy()
                df_2_common_synaptic_size = df_2_common["total_synapse_sizes_mean"].to_numpy()

                df_1_common_syn_density = df_1_common["syn_density"].to_numpy()
                df_2_common_syn_density = df_2_common["syn_density"].to_numpy()


                binary_conversion_pearson = find_pearson(df_1_common_binary_conversion, df_2_common_binary_conversion)
                binary_conversion_cosine = find_cosine(df_1_common_binary_conversion, df_2_common_binary_conversion)
                conversion_pearson = find_pearson(df_1_common_conversion, df_2_common_conversion)
                conversion_cosine = find_cosine(df_1_common_conversion, df_2_common_conversion)
                density_pearson = find_pearson(df_1_common_density, df_2_common_density)
                density_cosine = find_cosine(df_1_common_density, df_2_common_density)
                syn_volume_mean_pearson = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
                syn_volume_mean_cosine = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)
                syn_density_pearson = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
                syn_density_cosine = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)




                ####reset the df_1_common and df_1_common to reuse code
                df_1_common = df_1_common_converted
                df_2_common = df_2_common_converted

                if (not df_1_common.to_numpy().any()) or (not df_2_common.to_numpy().any()):
                    #print("none_in_converted")
                    pass
                else:
                    df_1_common_conversion = df_1_common["total_contact_conversion"].to_numpy()
                    df_2_common_conversion = df_2_common["total_contact_conversion"].to_numpy()

                    df_1_common_binary_conversion = np.copy(df_1_common_conversion)
                    df_2_common_binary_conversion = np.copy(df_2_common_conversion)


                    df_1_common_binary_conversion[df_1_common_binary_conversion>0] = 1.0
                    df_2_common_binary_conversion[df_2_common_binary_conversion>0] = 1.0

                    df_1_common_density = df_1_common["total_contact_density"].to_numpy()
                    df_2_common_density = df_2_common["total_contact_density"].to_numpy()


                    df_1_common_synaptic_size = df_1_common["total_synapse_sizes_mean"].to_numpy()
                    df_2_common_synaptic_size = df_2_common["total_synapse_sizes_mean"].to_numpy()

                    df_1_common_syn_density = df_1_common["syn_density"].to_numpy()
                    df_2_common_syn_density = df_2_common["syn_density"].to_numpy()

                    

                    binary_conversion_pearson_converted = find_pearson(df_1_common_binary_conversion, df_2_common_binary_conversion)
                    binary_conversion_cosine_converted = find_cosine(df_1_common_binary_conversion, df_2_common_binary_conversion)
                    conversion_pearson_converted = find_pearson(df_1_common_conversion, df_2_common_conversion)
                    conversion_cosine_converted = find_cosine(df_1_common_conversion, df_2_common_conversion)
                    density_pearson_converted = find_pearson(df_1_common_density, df_2_common_density)
                    density_cosine_converted = find_cosine(df_1_common_density, df_2_common_density)
                    syn_volume_mean_pearson_converted = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
                    syn_volume_mean_cosine_converted = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)
                    syn_density_pearson_converted = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
                    syn_density_cosine_converted = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)



            corr_dict["binary_conversion_pearson"] = binary_conversion_pearson
            corr_dict["binary_conversion_cosine"] = binary_conversion_cosine
            corr_dict["conversion_pearson"] = conversion_pearson
            corr_dict["conversion_cosine"] = conversion_cosine
            corr_dict["density_pearson"] = density_pearson
            corr_dict["density_cosine"] = density_cosine
            corr_dict["syn_volume_mean_pearson"] = syn_volume_mean_pearson
            corr_dict["syn_volume_mean_cosine"] = syn_volume_mean_cosine
            corr_dict["syn_density_pearson"] = syn_density_pearson
            corr_dict["syn_density_cosine"] = syn_density_cosine

            corr_dict["binary_conversion_pearson_converted"] = binary_conversion_pearson_converted
            corr_dict["binary_conversion_cosine_converted"] = binary_conversion_cosine_converted
            corr_dict["conversion_pearson_converted"] = conversion_pearson_converted
            corr_dict["conversion_cosine_converted"] = conversion_cosine_converted
            corr_dict["density_pearson_converted"] = density_pearson_converted
            corr_dict["density_cosine_converted"] = density_cosine_converted
            corr_dict["syn_volume_mean_pearson_converted"] = syn_volume_mean_pearson_converted
            corr_dict["syn_volume_mean_cosine_converted"] = syn_volume_mean_cosine_converted
            corr_dict["syn_density_pearson_converted"] = syn_density_pearson_converted
            corr_dict["syn_density_cosine_converted"] = syn_density_cosine_converted



            total_correlations.append(corr_dict)


        #write all of the dictionaries to the database
        self.insert(total_correlations,skip_duplicates=True)





In [5]:
start_time = time.time()
ContactCorrelation.populate()
print(f"Total time: {time.time()-start_time}")

Total time: 6296.99595284462


In [6]:
ContactCorrelation()

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b id of the postsynaptic neuron,n_seg_a n_presyns contacting onto segment_id,n_seg_b n_presyns contacting onto segment_b,n_seg_shared n_presyns contacting onto both segment_id and segment_b,n_seg_shared_converted n_presyns contacting onto both and converting on at least 1 postsyn,n_seg_union n_presyns contacting either segment_id or segment_b,binary_conversion_pearson pearson correlation for binary n_synapse/n_contact rate,binary_conversion_cosine cosine similarity correlation for binary n_synapse/n_contact rate,conversion_pearson Pearson correlation for n_synapse/n_contact rate,conversion_cosine cosine similarity for n_synapse/n_contact rate,density_pearson Pearson correlation for n_synapse/postsyn_length rate,density_cosine cosine similarity for n_synapse/postsyn_length rate,syn_volume_mean_pearson Pearson correlation for mean of synaptic volume,syn_volume_mean_cosine cosine similarity for mean of synaptic volume,syn_density_pearson Pearson correlation for n_synapses*synapse_sizes_mean/postsyn_length rate,syn_density_cosine cosine similarity for n_synapses*synapse_sizes_mean/postsyn_length rate,binary_conversion_pearson_converted pearson correlation for binary n_synapse/n_contact rate for axon group with at least 1 conversion,binary_conversion_cosine_converted cosine similarity correlation for binary n_synapse/n_contact rate for axon group with at least 1 conversion,conversion_pearson_converted Pearson correlation for n_synapse/n_contact rate for axon group with at least 1 conversion,conversion_cosine_converted cosine similarity for n_synapse/n_contact rate for axon group with at least 1 conversion,density_pearson_converted Pearson correlation for n_synapse/postsyn_length rate for axon group with at least 1 conversion,density_cosine_converted cosine similarity for n_synapse/postsyn_length rate for axon group with at least 1 conversion,syn_volume_mean_pearson_converted Pearson correlation for mean of synaptic volume for axon group with at least 1 conversion,syn_volume_mean_cosine_converted cosine similarity for mean of synaptic volume for axon group with at least 1 conversion,syn_density_pearson_converted Pearson correlation for n_synapses*synapse_sizes_mean/postsyn_length rate for axon group with at least 1 conversion,syn_density_cosine_converted cosine similarity for n_synapses*synapse_sizes_mean/postsyn_length rate for axon group with at least 1 conversion
2,648518346341352006,648518346341352223,1420,1262,171,15,2511,0.0820552,0.125,0.0638036,0.102839,-0.00671485,0.0206681,0.0701855,0.0994539,0.0701855,0.0994539,-0.875,0.125,-0.710552,0.102839,-0.420466,0.0206681,-0.406452,0.0994539,-0.406452,0.0994539
2,648518346341352006,648518346341353019,1420,76,0,0,1496,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341352006,648518346341353186,1420,1420,119,6,2721,-0.0258621,0.0,-0.0258621,0.0,-0.0173294,0.0,-0.0215825,0.0,-0.0215825,0.0,-1.0,0.0,-1.0,0.0,-0.540649,0.0,-0.721201,0.0,-0.721201,0.0
2,648518346341352006,648518346341353574,1420,208,1,0,1627,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341352006,648518346341353607,1420,1759,249,24,2930,0.107236,0.153846,0.0496818,0.0937615,-0.0104966,0.0250892,0.0180388,0.0449948,0.0180388,0.0449948,-0.846154,0.153846,-0.754,0.0937615,-0.536339,0.0250892,-0.34113,0.0449948,-0.34113,0.0449948
2,648518346341352006,648518346341353788,1420,13,0,0,1433,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341352006,648518346341353883,1420,166,0,0,1586,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341352006,648518346341354048,1420,2,0,0,1422,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341352006,648518346341354962,1420,121,12,4,1529,-0.2,0.0,-0.187867,0.0,-0.198999,0.0,-0.122994,0.0,-0.12299